# PairCorr Side-Band Benchmark
Synthetic test: injects a 7 % side-band at $k_1=k_0(1+1/14)$ and runs the TORUS detection pipeline.

In [ ]:

import numpy as np, matplotlib.pyplot as plt
N, L = 1000, 100
k0 = 2*np.pi/(L/10)
k1 = k0*(1+1/14)
amp1 = 0.07
x = L*np.random.rand(N)
y = L*np.random.rand(N) + amp1*np.sin(k1*x)


In [ ]:

from scipy.spatial.distance import pdist
dists = pdist(np.column_stack([x, y]))
bins = np.linspace(0, L, 200)
g2, _ = np.histogram(dists, bins=bins)
g2 = g2 / g2.max()


In [ ]:

try:
    from scipy.signal.windows import hann
except ImportError:
    try:
        from scipy.signal import hann  # SciPy <1.2
    except ImportError:
        import numpy as np
        def hann(N): return np.hanning(N)
from scipy.signal import periodogram
freqs, power = periodogram(g2 * hann(len(g2)), scaling='spectrum')


In [ ]:

k0_idx = np.argmax(power)
k0_val = freqs[k0_idx]
k1_val = k0_val*(1+1/14)
k1_idx = np.argmin(abs(freqs - k1_val))
deltaP = 10*np.log10(power[k1_idx]/power[k0_idx])
print('k0 =', k0_val, 'k1 =', k1_val, 'ΔP =', deltaP, 'dB')
if abs(k1_val-freqs[k1_idx])/k1_val <= .03 and -45 < deltaP < -25:
    print('TORUS-POSITIVE')
else:
    print('TORUS-NEGATIVE')
